## Gait Video Study 
### Identifying frames with HSRs in each video for each cohort and trial to establish break points and also evaluate the corresponding HSR labelling via the ground truth available. Further, downsample with smoothing to define fixed shape of the input tensor for models. 
#### Remember to preserve the original count of frames in a single stride (before down sampling via smoothing) for each stride to add as an additional artificial feature later to add information about speed of the subject to the model

In [3]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [4]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data'
frame_path_merged = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\multi_view_merged_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

### Utility functions 

In [ ]:
#Saving the HSRframes.txt file to the hip_height_normalized\\ containing the final .csvs for analysis
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            print (glob.glob(path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'))
            try:
                if (not os.path.exists(merged_path+'\\'+video+'\\HSRframes.txt')):
                    HSR_frames_file = path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'+'\\HSRframes.txt'
                    shutil.copy(HSR_frames_file, merged_path+'\\'+video+'\\hip_height_normalized\\') 
                    print ('HSR for', video, 'copied')
                else:
                    print ('HSR for', video, 'exists')
            except Exception as e:
                print (e)

[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_214_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Proje

In [5]:
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            start_time = time.time()
            front_video = path+cohort+trial+'\\lower_body\\'+'Inked'+video+'_1_Trim'
            shutil.copy(file, merged_path+'\\'+video) 
            frames = glob.glob(merged_path+'\\'+video+'\\hip_height_normalized\\*.csv')
            temp = pd.DataFrame(columns = labels) #Dataframe to collect 40 features from all ~1500 frames of each video
            for frame in frames:
#                     print (frame)
                frame_csv = pd.read_csv(frame, index_col = 0)

NameError: name 'file' is not defined

### Downsample with smoothing to define fixed shape input tensor for models

In [ ]:
#Use mean with disjoint windows to downsample while smoothing 
#Make sure to preserve count of frames in a frame before smoothing to add as a feature 

